# Run workflow using API

In [1]:
from pathlib import Path

from rich import print as pprint

In [2]:
import logging

logging.basicConfig(level=logging.WARNING)
# Set to WARNING to see only warnings
# Set to INFO to see sparql queries
# Set to DEBUG to see raw results

## Setup session directory

In this directory files like PDB files are stored and a DuckDB database for meta data.

In [3]:
session_dir = Path("session1")
session_dir

PosixPath('session1')

## Search Uniprot for structures

In [4]:
from protein_detective.uniprot import Query
from protein_detective.workflow import search_structures_in_uniprot

In [5]:
query = Query(
    taxon_id="9606",
    reviewed=True,
    subcellular_location_uniprot="nucleus",
    subcellular_location_go="GO:0005634",  # Cellular component - Nucleus
    molecular_function_go="GO:0003677",  # Molecular function - DNA binding
)

In [6]:
nr_uniprot, nr_pdbe, nr_af = search_structures_in_uniprot(query, session_dir, limit=80)
nr_uniprot, nr_pdbe, nr_af

(55, 13, 55)

Use [database queries](#query-session-database) to see what was found.

## Fetch structures from PDBe and Alphafold found in Uniprot


In [4]:
from protein_detective.workflow import retrieve_structures

In [5]:
download_path, nr_pdbs, nr_alphafolds = retrieve_structures(session_dir)
download_path, nr_pdbs, nr_alphafolds

Fetching Alphafold summaries: 100%|██████████| 55/55 [00:02<00:00, 25.34it/s]


(PosixPath('session1/downloads'), 13, 55)

## Filter out AlphafoldDB structures with low confidence

And write PDB files without those low confidence residues to new directory.

In [12]:
from protein_detective.alphafold.density import DensityFilterQuery
from protein_detective.workflow import density_filter

In [13]:
dquery = DensityFilterQuery(
    confidence=70.0,
    min_threshold=100,
    max_threshold=500,
)

In [14]:
result = density_filter(session_dir, dquery)
pprint(result)

DensityFilterSessionResult(density_filtered_dir=PosixPath('session1/density_filtered'),
                           nr_kept=24,
                           nr_discarded=31)


## Prune PDBe files
Prune the PDB files to only keep the first chain of the found Uniprot entries and rename that chain to A.

In [1]:
from protein_detective.workflow import prune_pdbs

In [7]:
(single_chain_dir, nr_passed) = prune_pdbs(session_dir)
(single_chain_dir, nr_passed)

Saving single chain PDB files from PDBe: 100%|██████████| 13/13 [00:02<00:00,  5.76it/s]


(PosixPath('session1/single_chain'), 13)

## Run powerfit

In [4]:
from protein_detective.powerfit.options import PowerfitOptions
from protein_detective.workflow import powerfit_commands

In [5]:
options = PowerfitOptions(
    target=Path("../../powerfit-tutorial/ribosome-KsgA.map"),
    resolution=13,
    angle=20,
    laplace=True,
)

In [6]:
commands, powerfit_run_id = powerfit_commands(session_dir, options)

In [7]:
powerfit_run_id

10

In [8]:
cwd = str(Path.cwd())
rel_commands = [c.replace(cwd, ".") for c in commands]
pprint(rel_commands)

[
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb4ne5.ent_B2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb4ne5.ent_B2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb7xho.ent_X2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb7xho.ent_X2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb7r5s.ent_X2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb7r5s.ent_X2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb4ndy.ent_B2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb4ndy.ent_B2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb4ne3.ent_B2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb4ne3.ent_B2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb7ywx.ent_X2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb7ywx.ent_X2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb4e45.ent_B2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb4e45.ent_B2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb7xhn.ent_X2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb7xhn.ent_X2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb4drb.ent_J2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb4drb.ent_J2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb4ne1.ent_B2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb4ne1.ent_B2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb4e44.ent_B2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb4e44.ent_B2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb4dra.ent_E2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb4dra.ent_E2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/single_chain/A8MT69_pdb4ne6.ent_B2A.pdb 
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/A8MT69_pdb4ne6.ent_B2A 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/density_filtered/AF-A0A087WUV0-F1-model_v4.pdb
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/AF-A0A087WUV0-F1-model_v4 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/density_filtered/AF-A0A0U1RQI7-F1-model_v4.pdb
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/AF-A0A0U1RQI7-F1-model_v4 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.map 13 ./session1/density_filtered/AF-A0A1B0GTS1-F1-model_v4.pdb
--laplace --resampling-rate 2 --num 10 --nproc 1 --directory ./session1/powerfit/10/AF-A0A1B0GTS1-F1-model_v4 
--delimiter , --angle 20',
    'powerfit ./session1/powerfit/10/ribosome-KsgA.

These commands should be run on a cluster. Here we will just run 2 of them.

In [9]:
!{rel_commands[6]}

Target file read from:                                                          
/home/verhoes/git/protein-detective/protein-detective/docs/session1/powerfit/10/
ribosome-KsgA.map                                                               
Target resolution: 13.00                                                        
Initial shape of density: 128 128 128                                           
Shape after trimming: 60 73 67                                                  
Shape after extending: 60 75 70                                                 
Template file read from:                                                        
/home/verhoes/git/protein-detective/protein-detective/docs/session1/single_chain
/A8MT69_pdb4e45.ent_B2A.pdb                                                     
Reading in rotations.                                                           
Requested rotational sampling density: 20.00                                    
Real rotational sampling den

In [10]:
!{rel_commands[0]}

Target file read from:                                                          
/home/verhoes/git/protein-detective/protein-detective/docs/session1/powerfit/10/
ribosome-KsgA.map                                                               
Target resolution: 13.00                                                        
Initial shape of density: 128 128 128                                           
Shape after trimming: 60 73 67                                                  
Shape after extending: 60 75 70                                                 
Template file read from:                                                        
/home/verhoes/git/protein-detective/protein-detective/docs/session1/single_chain
/A8MT69_pdb4ne5.ent_B2A.pdb                                                     
Reading in rotations.                                                           
Requested rotational sampling density: 20.00                                    
Real rotational sampling den

In [12]:
!{rel_commands[-1]}

Target file read from:                                                          
/home/verhoes/git/protein-detective/protein-detective/docs/session1/powerfit/10/
ribosome-KsgA.map                                                               
Target resolution: 13.00                                                        
Initial shape of density: 128 128 128                                           
Shape after trimming: 60 73 67                                                  
Shape after extending: 60 75 70                                                 
Template file read from:                                                        
/home/verhoes/git/protein-detective/protein-detective/docs/session1/density_filt
ered/AF-A8MT65-F1-model_v4.pdb                                                  
Reading in rotations.                                                           
Requested rotational sampling density: 20.00                                    
Real rotational sampling den

Once all powerfit jobs are done the results can be read into session database.
Then the best structure fit can be reported.

In [4]:
from protein_detective.workflow import powerfit_report

In [5]:
solutions = powerfit_report(session_dir)

In [6]:
len(solutions)

5238

In [8]:
pprint(solutions[0:5])

[
    PowerfitSolution(
        rank=1,
        cc=0.432,
        fish_z=0.463,
        rel_z=10.091,
        x=227.18,
        y=242.53,
        z=211.83,
        a11=0.0,
        a12=1.0,
        a13=0.0,
        a21=0.0,
        a22=0.0,
        a23=1.0,
        a31=1.0,
        a32=0.0,
        a33=0.0,
        powerfit_run_id=10,
        density_filter_id=None,
        af_id=None,
        uniprot_acc='A8MT69',
        pdb_id='4E45'
    ),
    PowerfitSolution(
        rank=1,
        cc=0.423,
        fish_z=0.452,
        rel_z=10.053,
        x=227.18,
        y=242.53,
        z=214.9,
        a11=0.0,
        a12=-0.0,
        a13=-1.0,
        a21=-0.604,
        a22=0.797,
        a23=0.0,
        a31=0.797,
        a32=0.604,
        a33=0.0,
        powerfit_run_id=10,
        density_filter_id=None,
        af_id=None,
        uniprot_acc='A8MT69',
        pdb_id='4NE5'
    ),
    PowerfitSolution(
        rank=1,
        cc=0.423,
        fish_z=0.452,
        rel_z=10.253,
        x=208.76,
        y=193.41,
        z=257.88,
        a11=-0.548,
        a12=0.816,
        a13=0.184,
        a21=0.632,
        a22=0.548,
        a23=-0.548,
        a31=-0.548,
        a32=-0.184,
        a33=-0.816,
        powerfit_run_id=10,
        density_filter_id=None,
        af_id=None,
        uniprot_acc='A8MT69',
        pdb_id='7XHO'
    ),
    PowerfitSolution(
        rank=2,
        cc=0.418,
        fish_z=0.445,
        rel_z=9.705,
        x=224.11,
        y=264.02,
        z=153.5,
        a11=0.797,
        a12=0.0,
        a13=0.604,
        a21=0.604,
        a22=-0.0,
        a23=-0.797,
        a31=0.0,
        a32=1.0,
        a33=-0.0,
        powerfit_run_id=10,
        density_filter_id=None,
        af_id=None,
        uniprot_acc='A8MT69',
        pdb_id='4E45'
    ),
    PowerfitSolution(
        rank=2,
        cc=0.414,
        fish_z=0.441,
        rel_z=9.815,
        x=227.18,
        y=138.15,
        z=227.18,
        a11=-0.0,
        a12=0.0,
        a13=1.0,
        a21=0.0,
        a22=-1.0,
        a23=0.0,
        a31=1.0,
        a32=0.0,
        a33=-0.0,
        powerfit_run_id=10,
        density_filter_id=None,
        af_id=None,
        uniprot_acc='A8MT69',
        pdb_id='4NE5'
    )
]